In [ ]:
# @title 7. Validate Model Performance (Quality Gate)

# CELL 1 [TAG: parameters]
# ---------------------------------------------------------
# Default parameters (Airflow will OVERWRITE these)
# ---------------------------------------------------------
INPUT_REAL_DATA = "s3://models/grouped_segments.pkl"
INPUT_MODEL_DIR = "s3://models/prod/"
OUTPUT_METRICS_PATH = "s3://models/prod/validation_metrics.json"
OUTPUT_PLOT_PATH = "s3://models/prod/validation_plot.png"
VEHICLE_TYPE = 1  # Motorcycle
MAX_RMSE_THRESHOLD = 0.15  # Fail if error > 15%

# MinIO Credentials
MINIO_ENDPOINT = "http://localhost:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password123"


In [ ]:
# CELL 2: Imports
import pickle
import json
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import s3fs
from scipy.stats import entropy


In [ ]:
# CELL 3: MinIO Configuration
fs = s3fs.S3FileSystem(
    key=MINIO_ACCESS_KEY,
    secret=MINIO_SECRET_KEY,
    client_kwargs={'endpoint_url': MINIO_ENDPOINT}
)


In [ ]:
# CELL 4: Helper Functions (Physics & Generation)
def calculate_vsp(speed_ms, acc_ms2, veh_type=1):
    """Calculates VSP using MOVESTAR coefficients."""
    if veh_type == 1:  # Motorcycle
        A, B, C, M, f = 0.0251, 0.0, 0.000315, 0.285, 0.285
    else:
        A, B, C, M, f = 0.156461, 0.002002, 0.000493, 1.4788, 1.4788

    vsp = (A * speed_ms) + (B * speed_ms**2) + (C * speed_ms**3) + (M * speed_ms * acc_ms2)
    return vsp / f

def get_distribution(vsp_data, bins):
    counts, _ = np.histogram(vsp_data, bins=bins)
    total = np.sum(counts)
    return counts / total if total > 0 else np.zeros(len(counts))

def generate_synthetic_sequence(matrix, states, length):
    """Generates a random walk using the trained matrix."""
    if matrix is None or length <= 0:
        return np.array([])

    current_idx = np.random.randint(0, len(matrix))
    path = [current_idx]

    for _ in range(length - 1):
        probs = matrix[current_idx]
        if np.sum(probs) == 0:
            next_idx = current_idx
        else:
            next_idx = np.random.choice(len(probs), p=probs)
        path.append(next_idx)
        current_idx = next_idx

    # Decode state indices to Speed values (Column 0)
    return states[path, 0]


In [ ]:
# CELL 5: Load Data & Models
print("Loading artifacts...")
model_base = INPUT_MODEL_DIR.rstrip("/")
try:
    # 1. Real Data (Grouped Segments)
    with fs.open(INPUT_REAL_DATA, 'rb') as f:
        grouped_segments = pickle.load(f)

    # 2. Models
    with fs.open(f"{model_base}/transition_matrices.pkl", 'rb') as f:
        trans_matrices = pickle.load(f)
    with fs.open(f"{model_base}/state_definitions.pkl", 'rb') as f:
        state_defs = pickle.load(f)

    print("✅ Loaded real data and models.")
except Exception as e:
    print(f"❌ Error loading artifacts: {e}")
    raise


In [ ]:
# CELL 6: Generate & Compare (The Heavy Lifting)
bins = np.arange(-20, 40, 2)
metrics = {}
traffic_labels = {0: "Heavy Traffic", 1: "Light Traffic"}

plt.figure(figsize=(14, 6))
for group_idx in range(2):
    print(f"\n--- Validating Group {group_idx} ({traffic_labels[group_idx]}) ---")

    # A. Prepare Real Data
    real_segments = grouped_segments[group_idx]
    if not real_segments:
        print("Skipping: No real data found.")
        continue

    real_concat = np.concatenate(real_segments)
    real_speed = real_concat[:, 0] / 3.6  # km/h -> m/s
    real_acc = real_concat[:, 1]          # Already m/s^2 from Step 0 fusion

    # B. Generate Synthetic Data
    target_duration = len(real_speed)
    syn_speed_kmh = generate_synthetic_sequence(trans_matrices[group_idx], state_defs[group_idx], target_duration)

    if len(syn_speed_kmh) == 0:
        print("Skipping: Model failed to generate data.")
        continue

    syn_speed = syn_speed_kmh / 3.6
    syn_acc = np.gradient(syn_speed)

    # C. Calculate VSP
    vsp_real = calculate_vsp(real_speed, real_acc, VEHICLE_TYPE)
    vsp_syn = calculate_vsp(syn_speed, syn_acc, VEHICLE_TYPE)

    # D. Compare Distributions
    dist_real = get_distribution(vsp_real, bins)
    dist_syn = get_distribution(vsp_syn, bins)

    # E. Compute Metric (RMSE)
    rmse = np.sqrt(np.mean((dist_real - dist_syn)**2))

    metrics[traffic_labels[group_idx]] = {
        "rmse": float(rmse),
        "sample_size_sec": int(target_duration)
    }

    print(f"RMSE Score: {rmse:.4f}")

    plt.subplot(1, 2, group_idx + 1)
    plt.bar(bins[:-1], dist_real, width=1.8, alpha=0.5, label='Real', color='blue')
    plt.plot(bins[:-1], dist_syn, color='red', linewidth=2, label='Synthetic Model')
    plt.title(f"{traffic_labels[group_idx]}\nRMSE: {rmse:.3f}")
    plt.xlabel("VSP Bin (kW/ton)")
    plt.legend()

plt.tight_layout()

# CELL 7: Evaluate Pass/Fail
print("\n=== FINAL EVALUATION ===")
print(json.dumps(metrics, indent=2))
max_error = max((m['rmse'] for m in metrics.values()), default=0.0)

# Save Plot
img_buf = io.BytesIO()
plt.savefig(img_buf, format='png')
img_buf.seek(0)
with fs.open(OUTPUT_PLOT_PATH, 'wb') as f:
    f.write(img_buf.getbuffer())

# Save Metrics
with fs.open(OUTPUT_METRICS_PATH, 'w') as f:
    json.dump(metrics, f)

# The Quality Gate
if max_error > MAX_RMSE_THRESHOLD:
    raise ValueError(
        f"❌ Model Validation Failed! Max RMSE ({max_error:.4f}) exceeds threshold ({MAX_RMSE_THRESHOLD})."
    )

print(f"✅ Model Validation Passed. (Max RMSE: {max_error:.4f})")
